In [1]:
import tensorflow as tf
import sklearn
import keras
from keras.layers import Dense, Dropout
from keras.models import Sequential
import sklearn.metrics
from sklearn.preprocessing import StandardScaler
import numpy as np
import matplotlib.pyplot as plt
import os
import h5py
import pickle
from collections import deque
import IPython
import time
import random
tf.__version__

Using TensorFlow backend.


'1.1.0'

In [2]:
if os.name == "nt":
    root = "E:/dataset"
else:
    root = "/media/ritchie46/5E061D60061D3A8B/dataset"

genres = ["hiphop", "edm_dance", "classical", "metal", "jazz"]

shape = (10,2584)

In [3]:
with open(os.path.join(root, "mfcc_scaler.pkl"), "rb") as f:
    scl = pickle.load(f)

In [4]:
if os.name == "nt":
    dset_f = h5py.File(os.path.join(r"C:\Users\vik\Desktop\empty", "mfcc_dset.hdf5"), "r")
else:
    dset_f = h5py.File(os.path.join(r"/home/ritchie46/Documents", "mfcc_dset.hdf5"), "r")

In [5]:
a = np.array(dset_f["labels"])
n_classes = np.unique(dset_f["labels"]).size

def create_class_weights(class_distru):
    total = sum(class_distru.values())
    new = dict()
    for i in classes:
        new[i] = total / classes[i]
        
    return new
    

classes = dict()

for i in range(n_classes):
    classes[i] = np.count_nonzero(a == i)
    
class_weights = create_class_weights(classes)
total = len(a)
print(n_classes, total, "\n", genres, "\n", class_weights)
del a


5 8366 
 ['hiphop', 'edm_dance', 'classical', 'metal', 'jazz'] 
 {0: 6.925496688741722, 1: 3.355796229442439, 2: 3.1239731142643765, 3: 5.325270528325907, 4: 20.110576923076923}


In [7]:
shuffle_seed = np.arange(0, total)
np.random.shuffle(shuffle_seed)

In [6]:

def generate_batch(batch_size, seed, test=False):  
    while True:
        if not test:
            print("newloop")
        data = deque()
        labels = deque()

        if test:
            seed_n = seed[6800:-3]
        else:
            seed_n = seed[:6800]
        c = 0
        np.random.shuffle(seed_n)
        
        for i in seed_n:
            x = dset_f["data"][i]
            y = dset_f["labels"][i]
            y = np.eye(n_classes)[y].reshape(n_classes)
            x = scl.transform(x.reshape(1, x.size))
            x = x.reshape(x.size)
            #x = x[]
            
            data.append(x.reshape(x.size))
            labels.append(y)

            if len(data) > batch_size:
                data.popleft()
                labels.popleft()            

            if (c + 1) % batch_size == 0:
                
                yield np.array(data).reshape(batch_size, x.size), np.array(labels)

            c += 1

            
def augment(batch_size, steps, test=False):
    """
    Augments the data passed through from the data generator.
    :param bath_size: (int) Maximum batch size is 1000. Choose the batch size so that
                            1000 % batch_size == 0.
    :param test: (bool): Augment testing data.
    """
    if test:
        gen = generate_batch(10, shuffle_seed, True)
    else:
        gen = generate_batch(10, shuffle_seed)
    while True:
        step = 0
        batch = next(gen)
        data = deque()
        labels = deque()
        x = batch[0]
        y = batch[1]
        
        
        for i in range(x.shape[0]):
            for j in range(100):
                data.append(x[i][j * 8:25000 + j * 8][::20])
                labels.append(y[i])
            
        data = np.array(data)
        labels = np.array(labels)
        
        seed_n = np.arange(0, data.shape[0])
        np.random.shuffle(seed_n)
        data = data[seed_n]
        labels = labels[seed_n]
        
        for i in range(batch_size):
            return_data = data[i * batch_size: (i + 1) * batch_size]
            if np.count_nonzero(return_data) > 1:
                step += 1
                yield return_data, labels[i * batch_size: (i + 1) * batch_size]
                
                if step == steps:
                    break



In [8]:
model = Sequential()
model.add(Dense(600, activation="relu", input_dim=1250, 
                kernel_regularizer=keras.regularizers.l2(0.01),
                kernel_initializer='glorot_uniform'))
model.add(Dropout(0.2))
model.add(Dense(600, activation="relu", input_dim=shape[1] * shape[0], 
                kernel_regularizer=keras.regularizers.l2(0.01)))
model.add(Dropout(0.2))
model.add(Dense(n_classes, activation="softmax"))

model.compile(optimizer=keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [9]:
gen = augment(100, 6790)
gen_test = augment(1000, 153, True)
epoch = 0

In [10]:
n = 2
model.fit_generator(gen, 6789, epochs=epoch + n, verbose=1, initial_epoch=epoch, 
                   validation_data=gen_test,
                   validation_steps=152,
                   class_weight=class_weights,
                   callbacks=[#keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=10, patience=200, verbose=0, mode='auto'),
                             keras.callbacks.TensorBoard(log_dir=root + '/Graph_1/run_mlp_600_600', histogram_freq=0,  
          write_graph=False, write_images=False)])
epoch += n

newloopEpoch 1/2

 215/6789 [..............................] - ETA: 301s - loss: 15.8241 - acc: 0.7653 

KeyboardInterrupt: 

In [32]:
#gen_test = augment(1600 * 100, True)
gen_test2 = augment(100, True) 
a = next(gen_test2)
print(a[0].shape)
true = model.predict_classes(a[0])
#["hiphop", "edm_dance", "classical", "metal", "jazz"]
print("\n",sklearn.metrics.classification_report(true, np.argmax(a[1], 1)))
print(sklearn.metrics.accuracy_score(true, np.argmax(a[1], 1)))
del gen_test2
del a
del true

(1000, 1250)
  32/1000 [..............................] - ETA: 0s
              precision    recall  f1-score   support

          0       0.01      0.01      0.01       149
          1       0.00      0.00      0.00       108
          2       1.00      0.75      0.86       400
          3       0.77      1.00      0.87       232
          4       0.00      0.00      0.00       111

avg / total       0.58      0.53      0.55      1000

0.534


/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [25]:

import librosa
import scipy
audio = "mp3"
genres_d = os.listdir(os.path.join(root, audio))
for i in range(10):
    label = random.randint(0, len(genres) - 1)
    genre_current = genres[label]
    if genre_current in genres:
        nrs = os.listdir(os.path.join(root, audio, genre_current))
        choice = random.randint(0, len(nrs) - 1)
        a = IPython.display.Audio(filename=os.path.join(root, audio, genre_current, nrs[choice]))
        wav = os.path.join(root, "wav", genre_current, nrs[choice].replace("mp3", "wav"))
        sr, ampl = scipy.io.wavfile.read(wav)
        if ampl.shape[0] == 1323000:
            mfcc = np.load(os.path.join(root, "mfcc", genre_current, nrs[choice].replace("mp3", "wav.npy")))
            mfcc = mfcc.reshape((1, 10 * 2584))
            mfcc = scl.transform(mfcc)
            print(mfcc.shape[0])
            mfcc = mfcc[:,:25000][:,::20]
            label = np.eye(6)[label].reshape(1, 6)
            print(os.path.join(root, audio, genre_current, nrs[choice]))
            
            p = np.argmax(model.predict(mfcc, 1))
            print("Neural nets predection:", genres[p])
            IPython.display.display(a)


1
/media/ritchie46/5E061D60061D3A8B/dataset/mp3/classical/Luigi_Boccherini-Boccherini__String_Quintet_in_E_Major__G__275__III__Minuetto.mp3
Neural nets predection: classical


1
/media/ritchie46/5E061D60061D3A8B/dataset/mp3/classical/Orlande_de_Lassus-Psalmi_Davidis_poenitentiales__Psalmus_Primus_Poenitentialis__Domine__ne_in_furore_tuo_arguas_me_.mp3
Neural nets predection: classical


1
/media/ritchie46/5E061D60061D3A8B/dataset/mp3/hiphop/Boef-Habiba.mp3
Neural nets predection: hiphop


In [168]:
model.save(os.path.join(root, "mlp_360.h5"))